In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import os
import xml.etree.ElementTree as ET
import json
import xmltodict
import xml.etree.ElementTree as ET
import pickle
import json
from typing import List

# ~~1. 특허 데이터 오픈 API 읽어오기~~
- 공공데이터포털, "한국과학기술연구원_특허DB"

In [3]:
uri = "http://161.122.37.103:6736/openapi-data/service/thesis/"
encodingKey = "wJ%2FpsYnaSH4n0J%2B%2BtvxXVLG8SP5xctqn5M1rOIBUM5sC3qi4eMcrV3j35I6Z2CEaMy32X2FNuzf4hDEXPiIuXA%3D%3D"
decodingKey = "wJ/psYnaSH4n0J++tvxXVLG8SP5xctqn5M1rOIBUM5sC3qi4eMcrV3j35I6Z2CEaMy32X2FNuzf4hDEXPiIuXA=="

service_name_dict = {'논문조회서비스': 'GetThesisInfoService', 
                     '특허조회서비스': 'GetPatentInfoService'}

In [4]:
# response 반환
def getResponse(url, params):
    try:
        response = requests.get(url, params=params)
    except Exception as e:
        print(e)
        return
    return response

In [5]:
# 전체 데이터개수 반환
def getLastPageNum(response):
    soup =BeautifulSoup(response.text, 'xml')
    return int(soup.find("totalCount").text)

In [6]:
def createData(response):
    contents = response.json()
    return pd.DataFrame(contents['response']['body']['items'])

In [7]:
def getData(url, totalPageNum):
    df_list = []
    pageNum = 1
    params ={'serviceKey' : decodingKey, 'pageNo' : pageNum, 'numOfRows' : '1000', 'type' : 'json'}

    with tqdm(total=totalPageNum, desc=f"진행 상황") as pbar:
        for i in range(totalPageNum):
            df_list.append(createData(getResponse(url, params=params)))

            pageNum+=1
            params ={'serviceKey' : decodingKey, 'pageNo' : pageNum, 'numOfRows' : '1000', 'type' : 'json'}
            pbar.update(1)  # 진행 상황 업데이트
            
    return df_list

In [8]:
params ={'serviceKey' : encodingKey, 'pageNo' : 1, 'numOfRows' : '1000', 'type' : 'xml'}
target_uri = uri+service_name_dict['특허조회서비스']
target_uri = 'http://161.122.37.103:6736/openapi-data/service/patent/patentApplication?pageNo=1&applyDate=201411&serviceKey='+decodingKey
# 데이터 전체 개수 카운팅
totalDataLength = getLastPageNum(getResponse(target_uri, params))
totalPageNum = int(totalDataLength/1000)+1

df = pd.concat(getData(target_uri, totalPageNum), ignore_index=True)

AttributeError: 'NoneType' object has no attribute 'text'

# 2. USPTO 데이터

In [4]:
def read_pkl_file(filePath):
    with open(filePath, 'rb') as file:
        data = pickle.load(file)
    return data

In [5]:
patent_data_path = "./data/patent"
patent_file_list = os.listdir(patent_data_path)
len(patent_file_list)

1

In [6]:
for filename in tqdm(patent_file_list, desc=f"Read Patent File"):
    p_data: List[dict] = read_pkl_file(os.path.join(patent_data_path, filename))
    break

Read Patent File:   0%|          | 0/1 [00:09<?, ?it/s]


In [9]:
patent_df = pd.DataFrame()

for data in tqdm(p_data, desc=f"Read {filename} data"):  
    patent_name = data.get('bibliographic_information', {}).get('title_of_invention', None)
    code_information = data.get('classifications', {})
    cpc_ipc_code = code_information.get('main_or_locrano_class', None)
    edition = code_information.get('ipc_edition', None)
    patent_df = pd.concat([patent_df, pd.DataFrame([{'patent_title':patent_name, 'cpc_ipc_code': cpc_ipc_code, 'edition': edition}])])

Read 2003.pkl data: 100%|██████████| 187104/187104 [11:25<00:00, 272.81it/s]


In [11]:
len(patent_df), patent_df.tail()

(187104,
                                         patent_title  cpc_ipc_code edition
 0       Method and apparatus for pay-per-view system   [H04N  716]     [7]
 0  Method and apparatus for customized rendering ...   [H04N  900]     [7]
 0  Conditional access filter as for a packet vide...  [H04N  7167]     [7]
 0  System for distributing and handling electroni...   [H04N  545]     [7]
 0  Method and apparatus for lost connectivity rec...  [H04N  7173]     [7])

In [13]:
patent_df.to_csv("./data/output/2003_patent_titlt_code(241204,1).csv")

# 1. cpc코드

In [15]:
cpc_data_path = "./data/cpc_title"
cpc_file_list = [filename for filename in os.listdir(cpc_data_path) if filename.endswith("txt")]
len(cpc_file_list)

9

In [26]:
for filename in cpc_file_list:
    df = pd.read_csv(os.path.join(cpc_data_path, filename), sep='\t', names=['code','depth','title'])
    break
df

,code,depth,title
0,D,NaN,TEXTILES; PAPER
1,D01,NaN,NATURAL OR MAN-MADE THREADS OR FIBRES; SPINNING
2,D01B,NaN,MECHANICAL TREATMENT OF NATURAL FIBROUS OR FIL...
3,D01B1/00,0.0,Mechanical separation of fibres from plant mat...
4,D01B1/02,1.0,"Separating vegetable fibres from seeds, e.g. c..."
...,...,...,...
5688,D21J5/00,0.0,Manufacture of hollow articles by transferring...
5689,D21J7/00,0.0,Manufacture of hollow articles from fibre susp...
5690,D99,NaN,SUBJECT MATTER NOT OTHERWISE PROVIDED FOR IN T...
5691,D99Z,NaN,SUBJECT MATTER NOT OTHERWISE PROVIDED FOR IN T...
